# Set Your Data Free with Kafka - Debezium / CDC

In [ ]:
USE GamePlayDB;

In [ ]:
SELECT DB_NAME();

In [ ]:
TRUNCATE TABLE dbo.tb_GamePlay;

## Enable CDC

In [ ]:
SELECT * FROM sys.databases;

In [ ]:
IF EXISTS(SELECT 1 FROM sys.databases WHERE name = DB_NAME() AND is_cdc_enabled = 1)
BEGIN
  EXEC sys.sp_cdc_disable_db;  
END 

In [ ]:
-- enable cdc on database level
EXEC sys.sp_cdc_enable_db;

In [ ]:
-- enable cdc on table
EXEC sys.sp_cdc_enable_table @source_schema = N'dbo',
                             @source_name   = N'tb_GamePlay',
                             @role_name = NULL;

## Configure Debezium

When we have enabled CDC on the database as well as on the table we can now configure Debezium and the MS SQL Connector. To configure we can use `curl`, however I find it easier to use **Postman**, so the following is all based on **Postman**:

* Check what connectors are installed by executing `GET - http://127.0.0.1:8083/connector-plugins`. Browsing the result you should see the following connector: `io.debezium.connector.sqlserver.SqlServerConnector`.
* See if there are any configured connectors: `GET - http://127.0.0.1:8083/connectors`. That will probably not return anything for a new installation of Kafka.

To configure Debezium we use a `.json.` formatted configuration which we `POST` as a request. The configuraation looks like so:

``` json
{
  "name": "gameplay-connector",
  "config": {
      "connector.class": "io.debezium.connector.sqlserver.SqlServerConnector",
      "database.hostname": "<ip-address-of-sql>",
      "database.port": "1433",
      "database.user": "<uid>",
      "database.password": "<pwd>",
      "database.dbname": "GamePlayDB",
      "database.server.name": "GamePlayServer",
      "database.whitelist": "dbo.tb_GamePlay",
      "database.history.kafka.bootstrap.servers": "broker:29092",
      "database.history.kafka.topic": "dbhistory.gameplay"
  }
}
```

The configuration above is also included as a file at: `.\code\debezium\config\connector-config.json`.

Replace the valaues for `database.hostname`, `database.user`, and `database.password`, and you are good to go:

* Post a request to `POST - http://127.0.0.1:8083/connectors` with the above configuration as body of the request.
* Check the status of the connector you just enabled: `GET - http://127.0.0.1:8083/connectors/gameplay-connector/status`. Notice the name of the connector as per the config, `gameplay-connector`,  is part of the URL. Word of warning: wait like 30 seconds to a minute after you have enabled the connector before you check the status. That way you will catch any configuration issues.

You can now go to **Confluent Control Center** and see how a couple of new topics have been created. However, you will notice that there is still no topic for your table, (if the table is empty to begin with).

We can now go ahead and set up the rest.

## Alter Proc

We don't need a hook-point as we are CDC:ing, and the events will automatically be picked up by Debezium,

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_LogWager @UserID INT,
                                          @GameID INT,
                                          @WagerAmount bigint,
                                          @PayoutAmount bigint,
                                          @EventTime datetime2
AS
BEGIN
  IF(@EventTime IS NULL)
  BEGIN
    SET @EventTime = SYSUTCDATETIME();
  END 

  BEGIN TRY
    BEGIN TRAN
      INSERT INTO dbo.tb_GamePlay(UserID, GameID, WagerAmount, PayoutAmount, EventTime)
      VALUES(@UserID, @GameID, @WagerAmount, @PayoutAmount, @EventTime);
      --do some more tx "stuff" here
    COMMIT TRAN
                              
  END TRY
  BEGIN CATCH
    ROLLBACK TRAN;
  END CATCH

END  
GO

## Time for gameplay

In [ ]:
SELECT * FROM dbo.tb_GamePlay;

In [ ]:
SET NOCOUNT ON;
EXECUTE dbo.pr_EmulateGamePlay @Loops = 3;

In [ ]:
SELECT * FROM dbo.tb_GamePlay;

When you have executed the code go to **Confluent Control Center** and ensure you have data in the `wagers` topic.

## Tear it Down

In [ ]:
EXEC sys.sp_cdc_disable_db; 

In [ ]:
EXEC sys.sp_cdc_disable_table   @source_schema = 'dbo' ,   
                                @source_name = 'tb_GamePlay',  
                                @capture_instance = 'all' 

In [ ]:
TRUNCATE TABLE dbo.tb_GamePlay;